In [ ]:
# bot punter 

import threading
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'
import pygame  # it is important to import pygame after that
import time

#Pandas Settings
pd.options.display.float_format = '{:,.2f}'.format

## Estratégias

In [ ]:
def bhct(page, i, lista_hct, tam_lista_hct, liga_tipster):   
    
    # checando se a competição do dia passada faz parte da estratégia       
    for j in range(tam_lista_hct):
        if liga_tipster == lista_hct[j]:
            # daqui para baixo analisamos os jogos da liga que foi passada
            tam_jogos = len(page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')) 
            
            # analisando se os jogos se encaixam na estratégia
            # CONDIÇÃO "LIGA" ACEITA
            
            # varrendo os jogos da liga
            # para cada jogo vamos ver a condição de nao ter vencido nenhuma das ultimas 5 partidas no geral
            for k in range(tam_jogos):
                page1 = page
                cont_lay = 0
                # aqui vão ficar os detalhes do evento que será o array diferente de vazio se o investimento compensar
                # ou seja, se se encaixar na estratégia
                investimento = []

                # nome das equipes
                times = page1.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')[k].find('a').text.strip()
                data = page1.find('',{'id':'dateHeader'}).text.strip()
                
                # abrindo a página do jogo com as últimas partidas do time da casa no geral na liga
                partida = page1.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')[k].find('a').get('href')
                partida = 'https://tipsterarea.com/' + partida
                link = requests.get(partida)
                page1 = bs(link.text, 'html.parser')
                
                partida = page1.find_all('div',{'class':'gameStatLinkWrapper'})[0].find('a').get('href').split('league-home')[0]
                partida = 'https://tipsterarea.com' + partida
                link = requests.get(partida)
                page1 = bs(link.text, 'html.parser')
                
                # verificando as 5 últimas partidas
                for contador in range(3):
                    
                    # detalhes da partida
                    placar = page1.find_all('tbody')[1].find_all('tr')[contador].find_all('td')[3].text.strip()
                    
                    # posicao do mandante - para saber se o time em questão é mandante naquele jogo e facilitar
                    # o calculo das partias sem vencer no geral
                    try:
                        teste = page1.find_all('tbody')[1].find_all('tr')[contador].find_all('td')[2].find('span').get('class')[0]
      
                        gh = int(placar[0])
                        ga = int(placar[4])
                        
                        if gh <= ga:
                            cont_lay += 1
                        else:
                            break
                    except:
                        # entra aqui se o time em questão estiver jogando fora de casa e então muda o calculo
                        # dos gols para saber quem fez mais (mandante ou visitante) e assim saber quem venceu a partida
                        gh = int(placar[0])
                        ga = int(placar[4])
                        
                        if gh >= ga:
                            cont_lay += 1
                        else:
                            break

                if cont_lay == 3:
                    # CONDIÇÃO "NÃO TER VITÓRIA NAS ULTIMAS 5 PARTIDAS NO GERAL" ACEITA
                    
                    # ver condição odds home
                    odd = odds_home('https://www.oddsportal.com/soccer/england/league-one/burton-milton-keynes-dons-Iio8ZDb7/')
                    odd = float(odd)
                    if odd >= 1.7 and odd <= 8:
                        # CONDIÇÃO DE ODD ACEITA - ESTRATÉGIA ACEITA
                        investimento.append(liga_tipster)
                        investimento.append(times)
                        investimento.append(data)
                        investimento.append('Back Home Contra Tendência')
                        
                        # mostrando os jogos que se encaixaram na estratégia
                        print(investimento)
                        
     
            # achou a liga e trabalhou nela, nao tem mais que comparar, pula para a próxima
            break        

In [ ]:
def over_25(i, lista_o25, tam_lista_o25):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
def bhft(i):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
def under_25(i):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
url_odds = 'https://www.oddsportal.com/soccer/italy/serie-a/parma-lecce-z7KzrgjU/'
odd = odds_under(url_odds)
print(odd)

In [ ]:
def odds_home(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # pegando a odd de abertura do match odds - home
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(2)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ooh = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break
    
    driver.quit()
    
    return ooh

In [ ]:
def odds_draw(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    # pegando a odd do match odds
    
    # draw
    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # pegando a odd de abertura do match odds - draw
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(3)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ood = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

        
    driver.quit()
    
    return ood

In [ ]:
def odds_away(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    # pegando a odd do match odds
    
    # away
    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle': 
            
            # pegando a odd de abertura do match odds - away
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(4)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ooa = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break
        
    driver.quit()
    
    return ooa

In [ ]:
def odds_over(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)
    
    # clicando no mercado over e under
    driver.find_element_by_css_selector('.ul-nav > li:nth-child(5) > a:nth-child(1)').click()
    time.sleep(4)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    # clicando em over/under 2.5          
    driver.find_element_by_css_selector('div.table-container:nth-child(12) > div:nth-child(1) > strong:nth-child(1) > a:nth-child(1)').click()
    time.sleep(1)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
    # odd over 2.5
    for i in range(1, 20):  
        var = str(i)                          
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # odd de abertura de over 2.5               
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(3)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            opening_odd_over = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

    
    driver.quit()
    
    return opening_odd_over

In [ ]:
def odds_under(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)
    
    # clicando no mercado over e under
    driver.find_element_by_css_selector('.ul-nav > li:nth-child(5) > a:nth-child(1)').click()
    time.sleep(4)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    # clicando em over/under 2.5          
    driver.find_element_by_css_selector('div.table-container:nth-child(12) > div:nth-child(1) > strong:nth-child(1) > a:nth-child(1)').click()
    time.sleep(1)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
    # odd under 2.5
    for i in range(1, 20):  
        var = str(i)                          
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # odd de abertura de under 2.5               
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(4)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            opening_odd_under = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

    
    driver.quit()
    
    return opening_odd_under

In [ ]:
# Script principal

# passar todo dia para o bot o link com a data futura de jogos para ele analisar
data = '14-01-2020'
url = 'https://tipsterarea.com/matches/date-'+ data      
link = requests.get(url)
page = bs(link.text, 'html.parser')

# times do jeito que está escrito no oddsportal (até agora igual ao betaminic)
teams = ['']

# lista de todas as competições do betaminic do jeito que ta escrito no site tipster
leagues = ['Croatia - Prva HNL','Denmark - 1st Division','Germany - 2. Bundesliga','Austria - Second League','Germany - 3. Liga',
            'Australia - A-League','Sweden - Allsvenskan','Germany - Bundesliga','Switzerland - Challenge League',
            'England - Championship','Scotland - Championship','Poland - First League','Netherlands - Eerste Divisie',
            'Poland - Ekstraklasa','Norway - Eliteserien','Netherlands - Eredivisie','Japan - J. League Division 1','Japan - J. League Division 2',
            'Belgium - First Division A','Spain - Primera Division','Spain - Segunda Division','England - League One','England - League Two',
            'Romania - Liga I','Colombia - Categoria Primera A','Mexico - Primera Division','Portugal - Segunda Liga','France - Ligue 1',
            'France - Ligue 2','USA - Major League Soccer','France - Championnat National','England - National League','Norway - 1. Division',
            'Hungary - NB I','Ireland - Premier Division','England - Premier League','Russia - Premier League','Scotland - Premiership',
            'Portugal - Primeira Liga','Argentina - Primera B Nacional - Zone A','Argentina - Primera B Nacional - Zone B','Italy - Serie A',
            'Brazil - Serie A','Italy - Serie B','Brazil - Serie B','Switzerland - Swiss Super League','China - Super League','Greece - Super League',
            'Turkey - Super Lig','Sweden - Superettan','Argentina - Superliga','Denmark - Superliga','Austria - Bundesliga',
            'Finland - Veikkausliiga','Finland - Ykkonen']
tam_leagues = len(leagues)

# lista de ligas do back home contra tendencia
lista_hct = ['Ireland - Premier Division','China - Super League','Belgium - First Division A','Austria - Second League',
            'Hungary - NB I','Turkey - Super Lig','England - Premier League','Switzerland - Swiss Super League',
            'Scotland - Premiership','Greece - Super League','Denmark - Superliga','Netherlands - Eerste Divisie',
            'Norway - Eliteserien','Norway - 1. Division','Italy - Serie B','Netherlands - Eredivisie',
             'Spain - Segunda Division','Portugal - Primeira Liga','England - League One']
tam_lista_hct = len(lista_hct)

# lista de ligas do over 2.5 
lista_o25 = ['Spain - Primera Division','Romania - Liga I','Argentina - Superliga','Scotland - Premiership','Germany - Bundesliga',
            'Sweden - Allsvenskan','Portugal - Primeira Liga','Poland - Ekstraklasa','Turkey - Super Lig','Italy - Serie A',
            'Denmark - Superliga','Australia - A-League','Belgium - First Division A','Mexico - Primera Division',
            'Netherlands - Eredivisie','Japan - J. League Division 1','China - Super League','Brazil - Serie A',
            'USA - Major League Soccer','France - Ligue 1','Ireland - Premier Division',
            'England - Premier League','Switzerland - Swiss Super League']
tam_lista_o25 = len(lista_o25)

# lista de times do over 2.5
lista_times_o25 = ['Atvidabergs','FC Midtjylland','Arsenal Sarandi','Dundalk','Wisla Plock','Ruch Chorzow','Basel',
                  'Godoy Cruz Antonio Tomba','Portland Timbers','Montreal Impact','FC Dallas','Shimizu S-Pulse',
                  'Jagiellonia Bialystok','FC Voluntari','Rayo Vallecano','Quilmes','Vitoria BA','Atletico de Rafaela',
                  'Guangzhou RF','Guangzhou Evergrande','Orebro','Kasimpasa','Piast Gliwice','Thun','Dinamo Bucuresti','Viitorul Constanta',
                  'FC Nordsjaelland','Leon','Ross County','Motherwell','Evian Thonon','Lyonnais','PSV Eindhoven',
                  'AZ Alkmaar','Tottenham','Liverpool','Manchester City','Chelsea','Nacional','Lazio','Melbourne Victory',
                  'Melbourne City','Wellington Phoenix','RB Leipzig','Royal Excel Mouscron','Torino','Cagliari']
tam_times_o25 = len(lista_times_o25)

lista_times_hft = ['PAOK Thessaloniki','New York City','FC Midtjylland','Dundalk','Basel','Young Boys','Shamrock Rovers',
                  'Shandong Luneng','Shanghai SIPG','Jiangsu Suning','Corinthians SP','Lanus','FC Dallas','Flamengo RJ',
                  'Aalesunds','Rosenborg','Palmeiras SP','Gremio RS','Seattle Sounders','Santos SP','Sport Recife PE',
                  'Benevento','Start Kristiansand','Varbergs BoIS','Bodo Glimt','Estudiantes La Plata','Atlanta United',
                  'Real Salt Lake','Beijing Guoan','Molde','AIK Stockholm','Malmo','AEK Athens','Randers FC','Fenerbahce',
                  'Bursaspor','FK Rostov','FC Copenhagen','Dinamo Zagreb','Zenit Sankt Peterburg','Astra Giurgiu',
                  'NK Osijek','Viitorul Constanta','SV Ried','Red Bull Salzburg','Santos Laguna','Lens','Club Brugge',
                  'Erzgebirge Aue','Celtic','Dundee United','Montpellier','Almere City FC','De Graafschap','Lorient',
                  'PSV Eindhoven','Marseille','Ajax Amsterdam','SC Cambuur','Cardiff','Liverpool','Leicester','Oxford United',
                  'Wycombe','Fleetwood','Benfica','Manchester City','Barcelona','Athletic Bilbao','Porto','Nacional',
                  'Levadiakos','Juventus','Stuttgart',"Borussia M'gladbach",'Olympiacos Piraeus','OFI Crete','Lazio',
                  'Sydney FC','RB Leipzig','Jong Ajax','Royal Antwerp','Odds BK','Brest']
tam_times_hft = len(lista_times_hft)

lista_times_u25 = ['Brasil de Pelotas RS','Inter Turku','FC Lahti','Corinthians SP','D.C. United','Avai SC','RoPS Rovaniemi',
                  'Tenerife','Cadiz','Getafe','Gremio RS','Cruzeiro MG','Bahia BA','Vasco Da Gama RJ','Alcorcon','Parana Clube PR',
                  'Botafogo RJ','Huracan Buenos Aires','FC Tokyo','FC Ufa','Darmstadt','Nantes','Lens','Anderlecht','Paris FC',
                  'Sunderland','Birmingham','Ipswich','Bolton','Blackpool','Malaga','Atletico Madrid','Deportivo La Coruna',
                  'Leganes','Entella','Longford Town','Grenoble Foot 38','Cremonese','Santos SP','PFC Sochi']
tam_lista_u25 = len(lista_times_u25)

# competições do site tipster
tam_competicoes = len(page.find_all('table',{'class':'games'}))  
ligas_tipster = []

# checando se as competições do dia fazem parte das do betaminic
for i in range(tam_competicoes):
    # ligas do dia
    ligas_tipster.append(page.find_all('table',{'class':'games'})[i].find('thead').find_all('th')[1].text)

    for j in range(tam_leagues):
        if ligas_tipster[i] == leagues[j]:
            # verificando se os filtros das estratégias são atendidos
            bhct(page, i, lista_hct, tam_lista_hct, ligas_tipster[i])
            #over_25(i, lista_o25, tam_lista_o25)
            #bhft(i)
            #under_25(i)
            break

In [ ]:
nome = "times_beta2.txt"   
with open(nome, 'w', encoding='utf-8') as arq:
    # CORPO DO WITH
    for j in range(1301):
        arquivo = open('times_beta.txt')
        b = arquivo.read().split('\n')[j]
        print(b)
        if j < 1300:
            arq.write(b + '\n')
        else:
            arq.write(b)
    arquivo.close()

In [4]:
arquivo = open('times_beta.txt')
b = arquivo.read().split('\n')

arquivo.close()

arquivo = open('times_beta2.txt')
b = arquivo.read().split('\n')
print(b[1300])
arquivo.close()

Western United
